## What `Going Modular` mean?

We are going to turn a **Python Notebook** code into a series of Python's that offer similar functionalities.

For example we could turn our notebook into a sereis of Python files:
* `data_setup.py`: Download and prepare the data
* `engine.py`: Define some training functions
* `model.py`: Define the Model class
* `train.py`: Train the Model
* `utils.py`: Define some helpfull functions

The format that out Project we would like to have is the following:
```
pytorch_project/
├── pytorch_project/
│   ├── data_setup.py
│   ├── engine.py
│   ├── model.py
│   ├── train.py
│   └── utils.py
├── models/
│   ├── model_1.pth
│   └── model_2.pth
└── data/
    ├── data_folder_1/
    └── data_folder_2/
```

## Why want to `Go Modular`?

For large scale rojects creating and modifying Python files is must faster and eazier than having the entire Porject into a large Notebook.

Some `pros` for going modular are:
1. Controlling the version on dependencies
2. Can package some code together to save rewritting
3. No confusing text and images inside the code

## Best Practice

It's the best practice to _start_ the Project in a Notebook, understand the Data using Text and Images and then after having some knowledge of what you are going move the most usefull pieces of code to Python scripts. 

## Script Mode

**Script Mode** uses Jupyter's Notebook special commands to turn a cell into a Python Script. For example the bellow cell is creating a Python Script for printing "Hello World"

In [1]:
%%writefile hello_world.py

print("Hello World")

Writing hello_world.py


We can run this file using:

In [2]:
!python hello_world.py

Hello World


As notice, the special command is `writefile`

## Creating a Folder for Storing the Python Scripts

In [3]:
from os import makedirs

# If the directory exists then it will leave it unharmed
makedirs("going_modular", exist_ok=True)

## Creating a Folder to Store the Dataset

In [4]:
from os import makedirs

# If the directory exists then it will leave it unharmed
makedirs("going_modular/data", exist_ok=True)

## Creating a Folder to Store the Python Scripts

In [5]:
from os import makedirs

# If the directory exists then it will leave it unharmed
makedirs("going_modular/going_modular", exist_ok=True)

## Creating a Folder to Store the Models

In [6]:
from os import makedirs

# If the directory exists then it will leave it unharmed
makedirs("going_modular/models", exist_ok=True)

## Downloading the Data

In [7]:
import requests
import zipfile
from pathlib import Path
from os import remove


# Initializing the Path Object
data_path = Path("/content/going_modular/data")
dataset_path = data_path / "pizza_steak_sushi"

# If the Path Object doesn't exists we want to create it
if dataset_path.is_dir():
    print(f"{dataset_path} already exists...")
else:
    print(f"{dataset_path} doesn't exists... Creating it...")

    dataset_path.mkdir(parents=True, exist_ok=True)

# Downloading the Dataset
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    req = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")

    print("Downloading the Dataset...")

    f.write(req.content) # Downloads the content of the request to the specified folder

# Unzipping the Dataset from Path
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping Dataset...")
    
    zip_ref.extractall(dataset_path) # Extracting the `zip file` to Path

# Deletting the `zip file`
remove(data_path / "pizza_steak_sushi.zip")

print(f"Dataset Downloaded Successfully on {dataset_path}")

/content/going_modular/data/pizza_steak_sushi doesn't exists... Creating it...
Unzipping Dataset...
Dataset Downloaded Successfully on /content/going_modular/data/pizza_steak_sushi


## Setting the Training and Testing Paths

In [8]:
train_path = dataset_path / "train"
test_path = dataset_path / "test"

print(train_path)
print(test_path)

/content/going_modular/data/pizza_steak_sushi/train
/content/going_modular/data/pizza_steak_sushi/test


## Creating Dataset and Data Loader Instances (Python Script)

In [9]:
%%writefile going_modular/going_modular/data_setup.py
from os import cpu_count
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Setting the Cores that will be used for the Data Loaders
NUM_WORKERS = cpu_count()

def create_dataloaders(train_dir: str, 
                       test_dir: str,
                       train_transform: transforms.Compose,
                       test_transform: transforms.Compose,
                       batch_size: int,
                       num_workers: int=NUM_WORKERS):

    # Creating the Datasets using `ImageFolder`
    train_ds = datasets.ImageFolder(root=train_dir, transform=train_transform)
    test_ds = datasets.ImageFolder(root=test_dir, transform=test_transform)

    # Getting the labels from the Dataset
    classes = train_ds.classes

    # Turn the Datasets into Dataloaders using `DataLoader`
    train_dl = DataLoader(dataset=train_ds,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=num_workers,
                          pin_memory=True) # Speed thing up when we (during training) moving data from CPU to GPU
    test_dl = DataLoader(dataset=test_ds,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=num_workers,
                         pin_memory=True)
    
    # Returning the Dataloaders and Labels
    return train_dl, test_dl, classes

Writing going_modular/going_modular/data_setup.py


## Creating Model (Python Script)

In [10]:
%%writefile going_modular/going_modular/model.py
import torch
from torch import nn

class TinyVGG(nn.Module):
    def __init__(self, input_size, hidden_units, output_size):
        super().__init__()

        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_size,
                      out_channels=hidden_units,
                      kernel_size=(3, 3),
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=(3, 3),
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2),
                         stride=2) # The default stride is the same as the kernel size
        ) # Output: (`batch_size`, `hidde_units`, 30, 30)

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=(3, 3),
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=(3, 3),
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2))
        ) # Output: (`batch_size`, `hidde_units`, 13, 13)

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*13*13, out_features=output_size),
        ) # Output: (`batch_size`, `output_size`)

    def forward(self, x):
        return self.classifier(self.conv_block_2(self.conv_block_1(x)))

Writing going_modular/going_modular/model.py


## Creating some Training and Evaluating Functions

In [11]:
%%writefile going_modular/going_modular/engine.py
import torch
from tqdm.auto import tqdm
from collections.abc import Callable
from timeit  import default_timer as timer


def accuracy_fn(model_logits, labels):
    labels_pred = torch.softmax(model_logits.type(torch.float32), dim=1).argmax(dim=1)
    return (torch.sum(labels_pred == labels).item() / len(labels)) * 100


def training_step(model: torch.nn.Module,
                  train_dl: torch.utils.data.DataLoader,
                  loss_fn: torch.nn.Module,
                  eval_metric: Callable[[torch.Tensor, torch.Tensor]],
                  optim: torch.optim.Optimizer,
                  n_batch_prints: int=None):
    
    batch_size = len(next(iter(train_dl))[0])
    model_device = next(model.parameters()).device
    train_loss, train_eval = 0, 0
    dummy = False # For styling the output

    model.train()
    for batch_num, (x_train, y_train) in enumerate(train_dl, start=1):
        # Moving Batches to Device
        x_train, y_train = x_train.to(model_device), y_train.to(model_device)

        model_logits = model(x_train)

        loss = loss_fn(model_logits, y_train)
        train_loss += loss.item()
        train_eval += eval_metric(model_logits, y_train)

        optim.zero_grad()
        loss.backward()
        optim.step()

        if n_batch_prints and (batch_num % (len(train_dl) // n_batch_prints) == 0):
            dummy = True
            print(f"\tLooked at {batch_num*batch_size}/{len(train_dl)*batch_size} training samples...")

    train_loss /= len(train_dl)
    train_eval /= len(train_dl)

    if dummy:
        print("-" * 107)

    return train_loss, train_eval


def validation_step(model: torch.nn.Module,
                    valid_dl: torch.utils.data.DataLoader,
                    loss_fn: torch.nn.Module,
                    eval_metric: Callable[[torch.Tensor, torch.Tensor]],
                    n_batch_prints: int=None):
    
    batch_size = len(next(iter(valid_dl))[0])
    model_device = next(model.parameters()).device
    valid_loss, valid_eval = 0, 0
    dummy = False # For styling the output

    model.eval()
    with torch.inference_mode():
        for batch_num, (x_valid, y_valid) in enumerate(valid_dl, start=1):
            # Moving Batches to Device
            x_valid, y_valid = x_valid.to(model_device), y_valid.to(model_device)

            model_logits = model(x_valid)

            valid_loss += loss_fn(model_logits, y_valid).item()
            valid_eval += eval_metric(model_logits, y_valid)

            if n_batch_prints and (batch_num % (len(valid_dl) // n_batch_prints) == 0):
                dummy = True
                print(f"\tLooked at {batch_num*batch_size}/{len(valid_dl)*batch_size} validation samples...")

        valid_loss /= len(valid_dl)
        valid_eval /= len(valid_dl)

        if dummy:
            print("-" * 107)

        return valid_loss, valid_eval


def fit(model: torch.nn.Module,
        epochs: int,
        train_dl: torch.utils.data.DataLoader,
        valid_dl: torch.utils.data.DataLoader,
        loss_fn: torch.nn.Module,
        eval_metric: Callable[[torch.Tensor, torch.Tensor]],
        optim: torch.optim.Optimizer,
        n_epoch_per_print: int=1,
        n_train_batch_prints: int=None,
        n_valid_batch_prints: int=None):
    
    start_time = timer()
    train_losses, train_evals = [], []
    valid_losses, valid_evals = [], []

    print("Starting Process...")
    
    for epoch in tqdm(range(1, epochs + 1)):
        # Training Step
        train_loss, train_eval = training_step(model, train_dl, loss_fn, eval_metric, optim, n_train_batch_prints)

        # Validation Step
        valid_loss, valid_eval = validation_step(model, valid_dl, loss_fn, eval_metric, n_valid_batch_prints)

        if (n_epoch_per_print > 0) and (epoch % n_epoch_per_print == 0):
            print(
                f"-> Epoch: {epoch} | "
                f"Train Loss: {train_loss:.4f} | "
                f"Train Accuracy: {train_eval:.2f}% | "
                f"Test Loss: {valid_loss:.4f} | "
                f"Test Evaluation (%): {valid_eval:.2f}%")
            print("-" * 107)
        
        train_losses.append(train_loss)
        train_evals.append(train_eval)
        valid_losses.append(valid_loss)
        valid_evals.append(valid_eval)

    print("Process Completed Successfully...")

    return {"model_train_loss": train_losses,
        "model_train_eval": train_evals,
        "model_valid_loss": valid_losses,
        "model_valid_eval": valid_evals,
        "model_name": model.__class__.__name__,
        "model_loss_fn": loss_fn.__class__.__name__,
        "model_evaluating_m": eval_metric.__name__,
        "model_optimizer": optim.__class__.__name__,
        "model_device": next(model.parameters()).device.type,
        "model_epochs": epochs,
        "model_time": timer() - start_time}

Writing going_modular/going_modular/engine.py


## Creating Function to Save the Model (Python Script)

In [13]:
%%writefile going_modular/going_modular/utils.py
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    
    # Create the target Path Object
    target_path = Path(target_dir)

    # Creating the Directory
    target_path.mkdir(parents=True, exist_ok=True)

    # `model_name` should end with '.pt' or '.pth'
    assert model_name.endswith(".pth") or model_name.endswith(".pt")

    # Create the Path Object for the saved model
    save_model_path = target_path / model_name

    print(f"Saving Model to: {save_model_path}")

    # Save the model's `state_dict`
    torch.save(obj=model.state_dict(),
               f = save_model_path)

    print(f"Model Successfully Saved to: {save_model_path}")

Writing going_modular/going_modular/utils.py


## Creating Function to Train and Evaluate the Model (Python Script)

In [14]:
%%writefile going_modular/going_modular/train.py
import torch
from torch import nn, optim
from torchvision import transforms
import data_setup, model, engine, utils
import os


# Setting up Hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LR = 1e-3

# Seting up Directories
train_dir = "/content/going_modular/data/pizza_steak_sushi/train"
test_dir = "/content/going_modular/data/pizza_steak_sushi/test"

# Setting up Device Agnostic Code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Creating the Transformations
train_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])
test_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor()
])

# Creating the DataLoaders using `data_setup.py`
train_dl, test_dl, classes = data_setup.create_dataloaders(train_dir=train_dir,
                                                            test_dir=test_dir,
                                                            train_transform=train_transform,
                                                            test_transform=test_transform,
                                                            batch_size=BATCH_SIZE)

# Creating the Model using `model.py`
modelv0 = model.TinyVGG(input_size=3,
                        hidden_units=HIDDEN_UNITS,
                        output_size=len(classes)).to(device)

# Setting up Loss Function and Optimizer
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(params=modelv0.parameters(), lr=LR)

# Training and Evaluating the Model using `engine.py`
res_0 = engine.fit(modelv0, NUM_EPOCHS, train_dl, test_dl, loss_fn, engine.accuracy_fn, opt)

# Saving the Model using `utils.py`
utils.save_model(model=modelv0,
                 target_dir="/content/going_modular/models",
                 model_name="modelv0.pth")

Writing going_modular/going_modular/train.py


Now our final directory structure looks like:
```
data/
  pizza_steak_sushi/
    train/
      pizza/
        train_image_01.jpeg
        train_image_02.jpeg
        ...
      steak/
      sushi/
    test/
      pizza/
        test_image_01.jpeg
        test_image_02.jpeg
        ...
      steak/
      sushi/
going_modular/
  data_setup.py
  engine.py
  model_builder.py
  train.py
  utils.py
models/
  saved_model.pth
```

## Running the `train.py` Python Script

In [15]:
!python going_modular/going_modular/train.py

Starting Process...
  0% 0/5 [00:00<?, ?it/s]-> Epoch: 1 | Train Loss: 1.1031 | Train Accuracy: 25.78% | Test Loss: 1.0960 | Test Evaluation (%): 39.96%
-----------------------------------------------------------------------------------------------------------
 20% 1/5 [00:03<00:15,  3.80s/it]-> Epoch: 2 | Train Loss: 1.0934 | Train Accuracy: 45.70% | Test Loss: 1.0988 | Test Evaluation (%): 26.04%
-----------------------------------------------------------------------------------------------------------
 40% 2/5 [00:07<00:10,  3.57s/it]-> Epoch: 3 | Train Loss: 1.1049 | Train Accuracy: 30.47% | Test Loss: 1.1063 | Test Evaluation (%): 27.08%
-----------------------------------------------------------------------------------------------------------
 60% 3/5 [00:10<00:06,  3.50s/it]-> Epoch: 4 | Train Loss: 1.0892 | Train Accuracy: 31.64% | Test Loss: 1.0765 | Test Evaluation (%): 31.25%
----------------------------------------------------------------------------------------------------